# Practice Problems based on Week 1 content

## Getting started + Julia basics

- Install Julia. Add the following packages: JuMP, IJulia, Clp, SCS, ECOS, NamedArrays.
- Create, name, and run an IJulia notebook. Save it as a PDF.

In [ ]:
### Uncomment if the packages have not been added

# using Pkg

# Pkg.add("JuMP")
# Pkg.add("Clp")
# Pkg.add("SCS")
# Pkg.add("ECOS")
# Pkg.add("NamedArrays")
# Pkg.add("Plots")

## LP Basics

- (**Q1**) Farmer Jane owns 45 acres of land. She is going to plant each with wheat or corn. Each acre planted with wheat yields \\$200 profit; each with corn yields \\$300 profit. The labor and fertilizer used for each acre are given in the table below. One hundred workers and 120 tons of fertilizer are available. How should Jane plant her crops to maximize profit?

    - Write down the full mathematical model.
    - Solve the problem using the graphical method.
    - Implement and solve the problem in Julia.
    
||Wheat|Corn|
|-----|-----|-----|
|Labor|3 Workers|2 Workers|
|Fertilizer|2 tons| 4 tons|

**<font color='brown'>Answer:</font>**

- **Mathematical Model**

    - Decision variables:
    
        - Wheat acreage $X_w$
        - Corn acreage $X_c$

    - Objective function:
    
        - $\max 200 \cdot X_w + 300 \cdot X_c$

    - Constraints:
    
        - C1: $X_w + X_c \leq 45$
        - C2: $3 \cdot X_w + 2\cdot X_c \leq 100$
        - C3: $2 \cdot X_w + 4\cdot X_c \leq 120$
        - C4: $X_w \geq 0$
        - C5: $X_c \geq 0$

- **Graphical method**

In [ ]:
using Plots

X_c = range(0,45,step=0.1)

labels = ["C1" "C2" "C3" "C4"]

c1 = 45 .- X_c
c2 = (100 .- (2*X_c))./3
c3 = (120 .- (4*X_c))./2
c4 = 0*X_c

#plot(Shape([(0,0),(0,100/3),(20,20),(30,0)]), line = 0, leg = false, fillcolor = plot_color(:yellow, 0.3))
plot(X_c,[c1,c2,c3,c4],label=labels,line=4)
plot!([0], seriestype="vline",label="C5",line=4)
plot!(xlabel="Corn",ylabel="Wheat",xlims=(-0.5,45),ylims=(-0.5,45))

In [ ]:
plot!(Shape([(0,0),(0,100/3),(20,20),(30,0)]), line = 0, leg = false, fillcolor = plot_color(:yellow, 0.1))
plot!(X_c,(100 .- (3*X_c))./2,line = 4,linestyle=:dot)
plot!([20],[20],marker=:d,markersize = 10,color=:red)
annotate!(30,20,"max = \$ 10.000,00")

- **Using Julia**

In [ ]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, Xw >=0)
@variable(m, Xc >=0)

@objective(m, Max, 200Xw+300Xc)

@constraint(m,c1,Xw+Xc<=45)
@constraint(m,c2,3Xw+2Xc<=100)
@constraint(m,c3,2Xw+4Xc<=120)

print(m)

JuMP.optimize!(m)

obj_value = JuMP.objective_value(m)
Xw = JuMP.value(Xw)
Xc = JuMP.value(Xc)

In [ ]:
println("Optimal Solutions:")
println("Xw = ", Xw)
println("Xx = ", Xc)
println("Y = ",obj_value)

- (**Q2**) For each of the three Top Brass implementations (Top Brass, Top Brass Modular, Top Brass Compact), re-implement the model to add a new trophy type: karate trophies. Karate trophies require 3 board feet of wood, 1 plaque, and 1 brass karate figure on top. Karate trophies are sold for \\$10 each. There are 750 brass karate figures available.  omment on how easy or hard it is to add a new trophy type in each case.

In [10]:
using JuMP, Clp

m2 = Model(Clp.Optimizer)

@variable(m2, 0<= f <=1000)
@variable(m2, 0<= s <=1500)
@variable(m2, 0<= k <=750)

@objective(m2, Max, 12f + 9s + 10k)

@constraint(m2,c1, 4f + 2s +3k <= 4800)
@constraint(m2,c2, f + s + k <= 1750)

JuMP.optimize!(m2)

println("Objective: ",JuMP.objective_value(m2))
println("f: ",JuMP.value(f))
println("s: ",JuMP.value(s))
println("k: ",JuMP.value(k))

Objective: 17700.0
f: 650.0
s: 1100.0
k: 0.0
Coin0506I Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Dual inf 31 (3)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Clp0032I Optimal objective 17700 - 2 iterations time 0.002


- (**Q3**) A very famous optimization problem is the so-called “Stigler Diet Problem,” which was created in 1945 by American economist and Nobel laureate George Stigler. Stigler published a paper on his study of the optimal diet, which minimizes total annual cost while meeting the recommended daily allowance (RDA) of several nutrients. In his paper, Stigler created a table of 77 different foods (although if you look at the data, you’ll note that one of the foods is “Plate”...not sure about that one) and their nutrient content for 9 nutrients: calories, protein, calcium, iron, vitamin A, thiamine, riboflavin, niacin, and ascorbic acid. To make the calculations easier, Stigler normalized the data so that the nutrients shown are the content for \\$1-worth of the given food. \\$1 could buy you a lot more in 1945! You can find a sample of the first few rows and columns of the table below. Stigler published this paper before linear programming algorithms had been invented, but Stigler was a very intelligent man and used “trial and error, mathematical insight, and agility,” to arrive at a diet costing only \\$39.93 per year. Stigler stated there was no reason to believe he had found the cheapest diet, and other combinations could prove more cost-effective. 

    - Formulate Stigler’s diet problem as a linear program and solve it to find the actual optimal solution.  Stigler’s original data is provided as “stigler.csv” on Canvas. Use the code snippet posted on Canvas as “stigler.jl” to import the data into arrays that will be much easier to work with. How does your cheapest diet compare in annual cost to Stigler’s? For the purposes of this exercise, a year is 365.25 days.
    
    - Now suppose you wanted a diet consisting only of foods you like to eat. Modify the stigler.csv file by deleting any rows you want, leaving only your favorite foods as options for your diet. Solve the diet problem again and compare your total annual cost to the cost you got in the previous problem. What foods make up your optimal diet now?
    
Table from the diet problem:


||kCalories|Protein (g)|Calcium (g)|Iron (mg)|$\cdots$|
|------|------|------|------|------|------|
|Wheat Flour (Enriched)|44.7|1411|2|365|$\cdots$|
|Macaroni|11.6|418|0.7|54|$\cdots$|
|Wheat Ceral (Enriched)|11.8|377|14.4|175|$\cdots$|
|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\ddots$|

In [11]:
# using Pkg

# Pkg.add("CSV")
# Pkg.add("DataFrames")

In [12]:
using DataFrames, CSV, NamedArrays

#Load the data file (ref: Boyd/263)
raw = CSV.read("stigler.csv");

# turn DataFrame into an array
diet_array = convert(Array,raw);

# the names of the DataFrame (header) are the nutrients
nutrients = names(raw[2:end]);

# create a list of foods from the diet array
foods = diet_array[2:end,1];# create a dictionary of the minimum daily requirements of each nutrient
min_daily_req = Dict(zip(nutrients,diet_array[1,2:end]));

# create a NamedArray that specifies how much of each nutrient each food provides
using NamedArrays
food_nutrient_matrix = diet_array[2:end,2:end]

# rows are foods, columns are nutrients
food_nutrient_array = NamedArray(food_nutrient_matrix, (foods, nutrients), ("foods","nutrients"))

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1260
┌ Info: Precompiling NamedArrays [86f7a689-2022-50b4-a561-43c23ac3c673]
└ @ Base loading.jl:1260
┌ Warning: `lastindex(df::AbstractDataFrame)` is deprecated, use `ncol(df)` instead.
│   caller = top-level scope at In[12]:8
└ @ Core In[12]:8
┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[12]:8
└ @ Core In[12]:8


77×9 Named Array{Any,2}
      foods ╲ nutrients │     Calories (1000)  …   Ascorbic Acid (mg)
────────────────────────┼────────────────────────────────────────────
Wheat Flour (Enriched)  │                44.7  …                    0
Macaroni                │                11.6                       0
Wheat Cereal (Enriched) │                11.8                       0
Corn Flakes             │                11.4                       0
Corn Meal               │                36.0                       0
Hominy Grits            │                28.6                       0
Rice                    │                21.2                       0
Rolled Oats             │                25.3                       0
White Bread (Enriched)  │                15.0                       0
Whole Wheat Bread       │                12.2                       0
Rye Bread               │                12.4                       0
Pound Cake              │                 8.0                     